# Predicting sex from brain rhythms with deep learning

Dans un premier temps, l'objectif est de reproduire le réseau décrit dans l'article original 

https://www.nature.com/articles/s41598-018-21495-7.pdf

In [91]:
import cv2
import numpy as np
from tensorflow import keras

from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from kerastuner.tuners import RandomSearch

import sklearn


from core.data import *

### Loading and formatting data 

In [25]:
raw_x_train, raw_y_train = load_x('data/x_train.h5'), load_y('data/y_train.csv')

Started loading file data/x_train.h5
Finished loading the file.
Started loading file data/y_train.csv
Finished loading the file.


In [46]:
# here, flatten x and convert to format (N, 7, 500, 1) and y_train in format (N, 2)

def format_data(x_input):
    x = flatten_x(x_input)
    x = reorder_nhwc(x)

    # here, resize x to format 24, 256, 1

    n, h, w, c = len(x), 24, 256, 1
    x_resized = np.zeros((n, h, w, c))

    # resize each element of x
    for i in range(len(x)):
        x_resized[i, :, :, 0] = cv2.resize(x[i, :, :, 0], dsize=(w, h))

    return x_resized

x_resized = format_data(raw_x_train)
y = np.argmax(flatten_y(raw_y_train, 40), axis=1)

print(x_resized.shape, y.shape)

(37840, 24, 256, 1) (37840,)


### Definition of the model

In [94]:
# need to convert 7 x 500 into 24 x 256 ? linear interpolation + subsampling
# Hint : use openCV resize

def get_model(hp):
    model = Sequential()
    # our model is based on convolutional NN + reLu + average pooling
    # cf cours pour la justification théorique
    model.add(layers.Conv2D(filters=hp.Int('units', min_value=50, max_value=200, step=50), 
                            kernel_size=(3, 3), activation='relu', data_format='channels_last', input_shape=(h, w, c)))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=0.25))

    model.add(layers.Conv2D(filters=hp.Int('units', min_value=50, max_value=200, step=50),
                            kernel_size=(3, 3), activation='relu', data_format='channels_last'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=0.25))

    model.add(layers.Conv2D(filters=hp.Int('units', min_value=100, max_value=400, step=50),
                            kernel_size=(3, 3), activation='relu', data_format='channels_last', input_shape=(h, w, c)))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=0.25))

    model.add(layers.Conv2D(filters=hp.Int('units', min_value=100, max_value=400, step=50), kernel_size=(1, 7), activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(1, 2)))
    model.add(layers.Dropout(rate=0.25))

    model.add(layers.Conv2D(filters=hp.Int('units', min_value=50, max_value=200, step=50), kernel_size=(1, 3), activation='relu'))
    model.add(layers.Conv2D(filters=hp.Int('units', min_value=50, max_value=200, step=50), kernel_size=(1, 3), activation='relu'))

    # 2 units for binary_crossentropy
    model.add(layers.Flatten())
    # model.add(layers.Dense(units=2, activation='softmax'))
    model.add(layers.Dense(units=1, activation='sigmoid'))
    
    model.compile(loss='mean_squared_error', # can be binary_crossentropy
              optimizer='sgd',
              metrics=['accuracy'])
    return model

In [99]:
tuner = RandomSearch(
    get_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='dreemmva')

In [97]:
class_weight = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y), y)
print('Class weight is ', class_weight)

Class weight is  [0.64179104 2.26315789]


In [ ]:
# model.fit(x_resized, y, epochs=10, batch_size=32, class_weight=class_weight)

tuner.search(x_resized, y, epochs=5)

Train on 37840 samples
Epoch 1/5


 5664/37840 [===>..........................] - ETA: 26:10 - loss: 0.3115 - accuracy: 0.343 - ETA: 22:18 - loss: 0.2859 - accuracy: 0.468 - ETA: 19:43 - loss: 0.2820 - accuracy: 0.500 - ETA: 19:22 - loss: 0.2879 - accuracy: 0.484 - ETA: 19:00 - loss: 0.2744 - accuracy: 0.525 - ETA: 18:57 - loss: 0.2723 - accuracy: 0.552 - ETA: 18:47 - loss: 0.2595 - accuracy: 0.584 - ETA: 18:29 - loss: 0.2546 - accuracy: 0.597 - ETA: 18:13 - loss: 0.2503 - accuracy: 0.611 - ETA: 18:10 - loss: 0.2446 - accuracy: 0.631 - ETA: 18:11 - loss: 0.2432 - accuracy: 0.642 - ETA: 18:09 - loss: 0.2360 - accuracy: 0.656 - ETA: 18:07 - loss: 0.2280 - accuracy: 0.670 - ETA: 18:03 - loss: 0.2219 - accuracy: 0.680 - ETA: 17:48 - loss: 0.2186 - accuracy: 0.691 - ETA: 17:34 - loss: 0.2210 - accuracy: 0.691 - ETA: 17:16 - loss: 0.2212 - accuracy: 0.694 - ETA: 16:58 - loss: 0.2181 - accuracy: 0.701 - ETA: 16:44 - loss: 0.2172 - accuracy: 0.702 - ETA: 16:28 - loss: 0.2158 - accuracy: 0.707 - ETA: 16:13 - loss: 0.2147 - accur

11360/37840 [========>.....................] - ETA: 12:01 - loss: 0.1945 - accuracy: 0.771 - ETA: 12:02 - loss: 0.1943 - accuracy: 0.771 - ETA: 12:01 - loss: 0.1949 - accuracy: 0.770 - ETA: 12:02 - loss: 0.1949 - accuracy: 0.770 - ETA: 12:01 - loss: 0.1954 - accuracy: 0.770 - ETA: 12:01 - loss: 0.1956 - accuracy: 0.770 - ETA: 12:00 - loss: 0.1957 - accuracy: 0.769 - ETA: 12:00 - loss: 0.1958 - accuracy: 0.769 - ETA: 11:59 - loss: 0.1957 - accuracy: 0.769 - ETA: 11:58 - loss: 0.1956 - accuracy: 0.769 - ETA: 11:57 - loss: 0.1957 - accuracy: 0.769 - ETA: 11:56 - loss: 0.1955 - accuracy: 0.769 - ETA: 11:55 - loss: 0.1960 - accuracy: 0.769 - ETA: 11:54 - loss: 0.1959 - accuracy: 0.769 - ETA: 11:54 - loss: 0.1956 - accuracy: 0.770 - ETA: 11:54 - loss: 0.1954 - accuracy: 0.770 - ETA: 11:54 - loss: 0.1956 - accuracy: 0.770 - ETA: 11:54 - loss: 0.1954 - accuracy: 0.770 - ETA: 11:55 - loss: 0.1956 - accuracy: 0.770 - ETA: 11:54 - loss: 0.1952 - accuracy: 0.770 - ETA: 11:53 - loss: 0.1950 - accur

17312/37840 [============>.................] - ETA: 9:53 - loss: 0.1938 - accuracy: 0.77 - ETA: 9:52 - loss: 0.1935 - accuracy: 0.77 - ETA: 9:52 - loss: 0.1934 - accuracy: 0.77 - ETA: 9:51 - loss: 0.1939 - accuracy: 0.77 - ETA: 9:51 - loss: 0.1940 - accuracy: 0.77 - ETA: 9:50 - loss: 0.1940 - accuracy: 0.77 - ETA: 9:49 - loss: 0.1940 - accuracy: 0.77 - ETA: 9:48 - loss: 0.1939 - accuracy: 0.77 - ETA: 9:47 - loss: 0.1937 - accuracy: 0.77 - ETA: 9:47 - loss: 0.1935 - accuracy: 0.77 - ETA: 9:46 - loss: 0.1934 - accuracy: 0.77 - ETA: 9:46 - loss: 0.1932 - accuracy: 0.77 - ETA: 9:45 - loss: 0.1931 - accuracy: 0.77 - ETA: 9:44 - loss: 0.1929 - accuracy: 0.77 - ETA: 9:43 - loss: 0.1931 - accuracy: 0.77 - ETA: 9:43 - loss: 0.1929 - accuracy: 0.77 - ETA: 9:42 - loss: 0.1930 - accuracy: 0.77 - ETA: 9:41 - loss: 0.1930 - accuracy: 0.77 - ETA: 9:40 - loss: 0.1932 - accuracy: 0.77 - ETA: 9:39 - loss: 0.1933 - accuracy: 0.77 - ETA: 9:38 - loss: 0.1931 - accuracy: 0.77 - ETA: 9:38 - loss: 0.1932 - ac

23264/37840 [=================>............] - ETA: 7:29 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:28 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:27 - loss: 0.1917 - accuracy: 0.77 - ETA: 7:26 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:26 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:25 - loss: 0.1918 - accuracy: 0.77 - ETA: 7:24 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:24 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:23 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:22 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:21 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:21 - loss: 0.1920 - accuracy: 0.77 - ETA: 7:20 - loss: 0.1920 - accuracy: 0.77 - ETA: 7:20 - loss: 0.1920 - accuracy: 0.77 - ETA: 7:19 - loss: 0.1920 - accuracy: 0.77 - ETA: 7:19 - loss: 0.1920 - accuracy: 0.77 - ETA: 7:18 - loss: 0.1921 - accuracy: 0.77 - ETA: 7:18 - loss: 0.1919 - accuracy: 0.77 - ETA: 7:17 - loss: 0.1921 - accuracy: 0.77 - ETA: 7:17 - loss: 0.1921 - accuracy: 0.77 - ETA: 7:16 - loss: 0.1922 - accuracy: 0.77 - ETA: 7:15 - loss: 0.1921 - ac

...

29344/37840 [======================>.......] - ETA: 5:05 - loss: 0.1745 - accuracy: 0.77 - ETA: 5:04 - loss: 0.1744 - accuracy: 0.77 - ETA: 5:03 - loss: 0.1743 - accuracy: 0.77 - ETA: 5:03 - loss: 0.1743 - accuracy: 0.77 - ETA: 5:02 - loss: 0.1743 - accuracy: 0.77 - ETA: 5:01 - loss: 0.1743 - accuracy: 0.77 - ETA: 5:01 - loss: 0.1742 - accuracy: 0.77 - ETA: 5:00 - loss: 0.1742 - accuracy: 0.77 - ETA: 4:59 - loss: 0.1741 - accuracy: 0.77 - ETA: 4:58 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:58 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:57 - loss: 0.1739 - accuracy: 0.77 - ETA: 4:56 - loss: 0.1739 - accuracy: 0.77 - ETA: 4:55 - loss: 0.1739 - accuracy: 0.77 - ETA: 4:55 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:54 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:53 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:53 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:52 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:51 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:51 - loss: 0.1740 - accuracy: 0.77 - ETA: 4:50 - loss: 0.1739 - ac

35296/37840 [==========================>...] - ETA: 2:57 - loss: 0.1730 - accuracy: 0.78 - ETA: 2:57 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:56 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:55 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:55 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:54 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:53 - loss: 0.1728 - accuracy: 0.78 - ETA: 2:53 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:52 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:51 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:51 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:50 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:49 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:49 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:48 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:47 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:46 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:46 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:45 - loss: 0.1729 - accuracy: 0.78 - ETA: 2:44 - loss: 0.1730 - accuracy: 0.78 - ETA: 2:44 - loss: 0.1730 - accuracy: 0.78 - ETA: 2:43 - loss: 0.1731 - ac

37840/37840 [==============================] - 801s 21ms/sample - loss: 0.1741 - accuracy: 0.7786
Train on 37840 samples
Epoch 1/5


 5696/37840 [===>..........................] - ETA: 18:59 - loss: 0.1346 - accuracy: 0.906 - ETA: 15:22 - loss: 0.1471 - accuracy: 0.828 - ETA: 14:17 - loss: 0.1469 - accuracy: 0.843 - ETA: 13:46 - loss: 0.1753 - accuracy: 0.812 - ETA: 13:35 - loss: 0.1760 - accuracy: 0.806 - ETA: 13:29 - loss: 0.2008 - accuracy: 0.770 - ETA: 13:17 - loss: 0.1933 - accuracy: 0.790 - ETA: 13:05 - loss: 0.1847 - accuracy: 0.804 - ETA: 12:56 - loss: 0.1834 - accuracy: 0.798 - ETA: 12:52 - loss: 0.1747 - accuracy: 0.809 - ETA: 12:47 - loss: 0.1769 - accuracy: 0.806 - ETA: 12:49 - loss: 0.1785 - accuracy: 0.804 - ETA: 12:48 - loss: 0.1782 - accuracy: 0.807 - ETA: 13:02 - loss: 0.1781 - accuracy: 0.805 - ETA: 13:21 - loss: 0.1770 - accuracy: 0.804 - ETA: 13:36 - loss: 0.1776 - accuracy: 0.804 - ETA: 13:43 - loss: 0.1819 - accuracy: 0.797 - ETA: 13:51 - loss: 0.1868 - accuracy: 0.789 - ETA: 14:04 - loss: 0.1834 - accuracy: 0.794 - ETA: 14:03 - loss: 0.1836 - accuracy: 0.793 - ETA: 14:05 - loss: 0.1838 - accur

 8288/37840 [=====>........................] - ETA: 12:06 - loss: 0.1908 - accuracy: 0.781 - ETA: 12:09 - loss: 0.1912 - accuracy: 0.780 - ETA: 12:11 - loss: 0.1915 - accuracy: 0.780 - ETA: 12:14 - loss: 0.1913 - accuracy: 0.780 - ETA: 12:16 - loss: 0.1915 - accuracy: 0.780 - ETA: 12:19 - loss: 0.1916 - accuracy: 0.780 - ETA: 12:21 - loss: 0.1917 - accuracy: 0.780 - ETA: 12:22 - loss: 0.1915 - accuracy: 0.780 - ETA: 12:25 - loss: 0.1921 - accuracy: 0.779 - ETA: 12:26 - loss: 0.1923 - accuracy: 0.779 - ETA: 12:28 - loss: 0.1925 - accuracy: 0.778 - ETA: 12:31 - loss: 0.1924 - accuracy: 0.778 - ETA: 12:33 - loss: 0.1927 - accuracy: 0.778 - ETA: 12:36 - loss: 0.1925 - accuracy: 0.778 - ETA: 12:39 - loss: 0.1923 - accuracy: 0.778 - ETA: 12:42 - loss: 0.1920 - accuracy: 0.779 - ETA: 12:45 - loss: 0.1920 - accuracy: 0.779 - ETA: 12:48 - loss: 0.1923 - accuracy: 0.778 - ETA: 12:51 - loss: 0.1920 - accuracy: 0.779 - ETA: 12:54 - loss: 0.1921 - accuracy: 0.779 - ETA: 12:56 - loss: 0.1920 - accur

In [55]:
raw_x_test = load_x('data/x_test.h5')

Started loading file data/x_test.h5
Finished loading the file.


In [74]:
x_test = format_data(raw_x_test)

print(x_test.shape)

raw_predictions = model.predict(x_test)
y_test = average_predictions(raw_predictions, nb_trials = 40)
print(y_test)

(37840, 24, 256, 1)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 